In [1]:
import os
os.getcwd()

'/Users/yashasuttangi/Developer/MachineLearning/Projects/CancerClassification/Cancer-classification-end-to-end-using-MLFlow-and-DVC/research'

In [2]:
os.chdir("../")

In [3]:
%pwd

'/Users/yashasuttangi/Developer/MachineLearning/Projects/CancerClassification/Cancer-classification-end-to-end-using-MLFlow-and-DVC'

In [4]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path 

In [5]:
from ChestCancerClassifer.utils.common import read_yaml, create_directories
from ChestCancerClassifer.constants import * 

In [6]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir]) ## Creating data ingestion folder 
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [7]:
import os
import zipfile
import gdown 
from ChestCancerClassifer import logger 
from ChestCancerClassifer.utils.common import get_size

In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config 
    
    ## Fetch data from the source URL 
    def download_data(self) -> str:
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")
            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix + file_id, zip_download_dir, quiet=False)
            logger.info(f"Downloaded data successfully!")
        except Exception as e:
            raise e
        
    ## Extract the zip file into the data directory
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            logger.info("Extracting zip file...")
            zip_ref.extractall(unzip_path)
            logger.info(f"Extracted zip file into dir: {unzip_path} and size")

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


Reading yaml file : config/config.yaml
[2026-01-23 09:48:22,562: INFO: common]: YAML file: config/config.yaml loaded successfully
Reading yaml file : params.yaml
[2026-01-23 09:48:22,563: INFO: common]: YAML file: params.yaml loaded successfully
[2026-01-23 09:48:22,564: INFO: common]: created directory at : artifacts
[2026-01-23 09:48:22,565: INFO: common]: created directory at : artifacts/data_ingestion
[2026-01-23 09:48:22,565: INFO: 1838134735]: Downloading data from https://drive.google.com/file/d/1gBfjc4cPNRNxeRc-hSfh4GRmG7umkaFu/view?usp=sharing into file artifacts/data_ingestion/data.zip


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1gBfjc4cPNRNxeRc-hSfh4GRmG7umkaFu
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1gBfjc4cPNRNxeRc-hSfh4GRmG7umkaFu&confirm=t&uuid=f31eb2c1-57d8-4a6c-934e-56a8e482f5f7
To: /Users/yashasuttangi/Developer/MachineLearning/Projects/CancerClassification/Cancer-classification-end-to-end-using-MLFlow-and-DVC/artifacts/data_ingestion/data.zip
100%|██████████| 125M/125M [00:03<00:00, 39.5MB/s] 

[2026-01-23 09:48:27,655: INFO: 1838134735]: Downloaded data successfully!
[2026-01-23 09:48:27,666: INFO: 1838134735]: Extracting zip file...


[2026-01-23 09:48:28,083: INFO: 1838134735]: Extracted zip file into dir: artifacts/data_ingestion and size
